In [ ]:
library(dplyr)
# For r2
library(performance)
# For bootstrap confidence intervals
library(nlme)
library(lme4)
# For model comparison
library(MuMIn)
# For get preditions
library(modelr)
# For rendering tables
library(reactablefmtr)
library(ggplot2)
# For effect sizes
library(effectsize)
library(emmeans)

In [ ]:
# Load dataset for adding predictions
source("./scripts/code_join_data_full_dataset.R")

In [ ]:
# Load functions

## Inference
source("./R/function_for_inference_anova_table.R")
source("./R/function_for_inference_tukey_tables.R")
source("./R/function_for_inference_emmeans_and_percentage_diff.R")

## Plots
source("./R/function_masomenos_plots.R")

__Q1: What is the relative influence of water and/or nutrient availability on
tropical dry forest seedling growth and biomass allocation__

# Load models

In [ ]:
models_q1 <- readRDS("./processed_data/models_q1.RData")

In [ ]:
# Model available
names(models_q1)

In [ ]:
# Step done for getting predictions from models for Q1
data_for_predictions <-
    data_for_models %>%

        rownames_to_column("id") %>%

        # Remove unused variables
        dplyr::select(id, spcode, treatment, nfixer, init_height)

# ANOVAs

All models show a significant two-way interaction between Nfixer and Treatment

In [ ]:
models_q1 %>%

    # Remove traits and mass fractions models
    purrr::list_modify("agr_log" = NULL, "agr" = NULL) %>%
    anova_table_tidy(., model_list = T)

# Post-Hoc

The main argument of the paper is:

_Results from our shade house experiment show that nutrient addition had a
stronger effect on tropical dry forest seedling biomass and growth than water
addition. Moreover, we demonstrate that N-fixing species have a higher capacity
to take advantage of increased resource availability, particularly for nutrients,
and to a lesser degree water..._


__however I think this is not true since there are not significant differences
when the same treatment are compared (i.e. plus nutrients nfixer vrs plus nutrients non-nfixer)__

## Total Biomass

### How I reported in the paper

In [ ]:
as_data_frame(emmeans(models_q1$total_biomass,
        pairwise ~ treatment|nfixer,
        adjust = "tukey"
        )$contrast)  %>%
        select(-c(df, SE)) %>%
        reactable()

### How it should be done

In [ ]:
as_data_frame(emmeans(models_q1$total_biomass,
        pairwise ~ treatment*nfixer,
        adjust = "tukey"
        )$contrast)  %>%
        select(-c(df, SE)) %>%
        reactable()


### Percentage of Change

In [ ]:
emmeans_table_tidy(models_q1$total_biomass,
                        formula = "treatment|nfixer",
                        grouping_var = "nfixer")

## RGR

### How I reported in the paper

In [ ]:
as_data_frame(emmeans(models_q1$rgr,
        pairwise ~ treatment|nfixer,
        adjust = "tukey"
        )$contrast)  %>%
        select(-c(df, SE)) %>%
        reactable()

### How it should be done

In [ ]:
as_data_frame(emmeans(models_q1$rgr,
        pairwise ~ treatment*nfixer,
        adjust = "tukey"
        )$contrast) %>%
        select(-c(df, SE)) %>%
        reactable()

### Percentage of Change

In [ ]:
emmeans_table_tidy(models_q1$rgr,
                        formula = "treatment|nfixer",
                        grouping_var = "nfixer")

## Root shoot ratio

### How I reported in the paper

In [ ]:
as_data_frame(emmeans(models_q1$root_shoot_ratio_log,
        pairwise ~ treatment|nfixer,
        adjust = "tukey"
        )$contrast)  %>%
        select(-c(df, SE)) %>%
        reactable()

### How it should be done

In [ ]:
as_data_frame(emmeans(models_q1$root_shoot_ratio_log,
        pairwise ~ treatment*nfixer,
        adjust = "tukey"
        )$contrast)  %>%
        select(-c(df, SE)) %>%
        reactable()

## Percentage of Change

In [ ]:
emmeans_table_tidy(models_q1$root_shoot_ratio_log,
                        formula = "treatment|nfixer",
                        grouping_var = "nfixer")

# Plots Biomass and growth boxplot

In [ ]:
# Get predictions
string <- c("models_q1")

data_pred_biomass_growth <-

        # Get models prediction
        gather_predictions(data_for_predictions,

                           # Return predictions of:
                            models_q1$rgr,
                            models_q1$total_biomass,
                            models_q1$root_shoot_ratio_log)  %>%

            # Get fitted values
            pivot_wider(names_from = model, values_from = pred) %>%
             rename_all(
              funs(

                # rename columns
                stringr::str_to_lower(.) %>%

                # Remove string from name and replace it with pred_
                stringr::str_replace(., c(string), "pred_") %>%

                # Remove dollar sing
                gsub("\\$", "", .))) %>%


        # Back transform log variables
        mutate(pred_root_shoot_ratio = exp(pred_root_shoot_ratio_log)) %>%

        # Select only pred variables
        dplyr::select(-c(init_height, pred_root_shoot_ratio_log))

In [ ]:
# Generate plot combinations
vars_q1 <-
  crossing(

    # Get all numeric variables to plot (all y)
    as_tibble(t(combn(dplyr::select(data_pred_biomass_growth, where(is.numeric)) %>% names, 1))),

    # Select factor variables to plot
    x_axis_var = dplyr::select(data_pred_biomass_growth, nfixer) %>%  names,
    group_var = dplyr::select(data_pred_biomass_growth, treatment) %>%  names)

In [ ]:
par(mfrow = c(2,2))
vars_q1 %>%
      # Gererate plots
      pmap( ~boxplot_plot_pmap(data = data_pred_biomass_growth,
                                y = !!sym(..1), x = !!sym(..2),
                                fill = !!sym(..3)))

## r2 models

In [ ]:
models_q1 %>%

    # Remove traits and mass fractions models
    purrr::list_modify("agr_log" = NULL, "agr" = NULL) %>%

    map(., r2) %>%
    unlist()